In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv', header=True, inferSchema=True)

In [0]:
data.show(50)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
|     44|   1.0|     0|
|     45|   2.0|     0|
|     46|   1.0|     0|
|     47|   1.0|     0|
|     48|   1.0|     0|
|     50|   1.0|     0|
|     51|   1.0|     0|
|     54|   1.0|     0|
|     55|   1.0|     0|
|     59|   2.0|     0|
|     61|   2.0|     0|
|     64|   1.0|     0|
|     67|   1.0|     0|
|     68|   1.0|     0|
|     69|   1.0|     0|
|     71|   1.0|     0|
|     72|   1.0|     0|
|     77|   2.0|     0|
|     79|   1.0|

In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [0]:
train, test = data.randomSplit([0.8,0.2])

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [0]:
model = als.fit(train)

In [0]:
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   3.0|    28|  0.8332668|
|      5|   2.0|    26|    1.16423|
|      5|   1.0|    13|  0.4798147|
|      5|   3.0|    16|  2.4268005|
|      6|   1.0|     6| 0.81828296|
|      2|   1.0|    19| 0.90482444|
|      4|   1.0|    19|  3.8335714|
|      6|   2.0|    19|  1.4467306|
|      6|   1.0|    15|  0.7146801|
|      2|   1.0|    17|   0.908973|
|      5|   1.0|     9|  0.8856025|
|      1|   1.0|     4|    2.49667|
|      4|   1.0|    23|  1.7070502|
|      2|   2.0|     7|  3.6789823|
|      3|   1.0|     7|  1.8292378|
|      2|   1.0|    25|0.012339592|
|      2|   4.0|    21|  2.8959603|
|      0|   1.0|    11|0.036694713|
|      7|   1.0|    14|  0.7715444|
|      4|   3.0|     2|   2.696611|
+-------+------+------+-----------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [0]:
rmse = evaluator.evaluate(predictions)

In [0]:
print('RMSE = ', rmse)

RMSE =  1.9052293323453275


In [0]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [0]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     20|    11|
|     23|    11|
|     36|    11|
|     50|    11|
|     59|    11|
|     62|    11|
|     70|    11|
|     76|    11|
|     82|    11|
+-------+------+



In [0]:
recomendations = model.transform(single_user)

In [0]:
recomendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     82|    11|  2.9431672|
|     59|    11|  1.8839897|
|     20|    11|  1.6516352|
|     23|    11|  1.4332242|
|     62|    11|  0.8163062|
|     76|    11|   0.634952|
|     50|    11|  0.3361579|
|      0|    11|0.036694713|
|     36|    11| 0.00994269|
|     70|    11| -1.4423968|
+-------+------+-----------+

